In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data_cleaned.csv')

In [3]:
# convert nan into -1.0
#data['Incident Zip'] = data['Incident Zip'].fillna(-1.0).astype(int)

In [4]:
data['Created AM/PM'] = data['Created Date'].str.split().str[2]

In [5]:
# create a dictionary to represent the parents-children relationship in data
d0 = {}
d0['name'] = 'New York City'
d0['value'] = data.shape[0]
d0['children'] = []

In [6]:
# level1: total times of complaints of each borough
level1 = data.groupby('Borough').size().reset_index(name = "Times")

In [7]:
# add summarized records in boroughs into d['children']
for i in range(len(level1)):
    d1 = {}
    d1['name'] = level1.iloc[i]['Borough']
    d1['value'] = level1.iloc[i]['Times']
    d1['children'] = []
    d0['children'].append(d1)

In [8]:
# sort in the order of the complaints times of each borough
d0['children'] = sorted(d0['children'], key=lambda x: x['value'], reverse=True)

In [9]:
# level2: total times of complaints of each descriptor with corresponding borough
level2 = data.groupby(['Borough', 'Descriptor']).size().reset_index(name = "Times")

In [10]:
# add summarized records in level2
# into 'children' of each borough in d['children']
for i in range(len(level2)):
    d2 = {}
    #if level2.iloc[i]['Descriptor'] == -1:
    #    d2['name'] = 'NaN'
    #else:
    d2['name'] = str(level2.iloc[i]['Descriptor'])
    d2['value'] = level2.iloc[i]['Times']
    d2['children'] = []
    
    # add records to corresponding borough
    for j in d0['children']:
        if j['name'] == level2.iloc[i]['Borough']:
            j['children'].append(d2)

In [11]:
# sort in the order of the complaints times of each zip code
for l1 in range(len(d0['children'])):
    d0['children'][l1]['children'] = sorted(d0['children'][l1]['children'], key=lambda x: x['value'], reverse=True)

In [12]:
# level3: total times of complaints of AM or PM with corresponding boroughs & descriptors
level3 = data.groupby(['Borough', 'Descriptor', 'Created AM/PM']).size().reset_index(name = "Times")

In [13]:
# add summarized records in level3
# into 'children' of each descriptor of each borough in d['children']
for i in range(len(level3)):
    d3 = {}
    d3['name'] = str(level3.iloc[i]['Created AM/PM'])
    d3['value'] = level3.iloc[i]['Times']
    
    # add records to corresponding descriptor
    for j in d0['children']:
        if j['name'] == level3.iloc[i]['Borough']:
            for k in j['children']:
                if k['name'] == level3.iloc[i]['Descriptor']:
                    k['children'].append(d3)

In [14]:
# sort in the order of the complaints times of AM or PM
for l1 in range(len(d0['children'])):
    for l2 in range(len(d0['children'][l1]['children'])):
        d0['children'][l1]['children'][l2]['children'] = sorted(d0['children'][l1]['children'][l2]['children'], key=lambda x: x['value'], reverse=True)

In [15]:
d0

{'children': [{'children': [{'children': [{'name': 'PM', 'value': 188},
      {'name': 'AM', 'value': 126}],
     'name': 'Clear Water With Other Particles (Use Comments) (QEZ)',
     'value': 314},
    {'children': [{'name': 'PM', 'value': 193}, {'name': 'AM', 'value': 117}],
     'name': 'Cloudy Or Milky Water (QB1)',
     'value': 310},
    {'children': [{'name': 'PM', 'value': 155}, {'name': 'AM', 'value': 109}],
     'name': 'Taste/Odor, Chemical (QA2)',
     'value': 264},
    {'children': [{'name': 'PM', 'value': 165}, {'name': 'AM', 'value': 93}],
     'name': 'unknown odor/taste in drinking water (QA6)',
     'value': 258},
    {'children': [{'name': 'PM', 'value': 160}, {'name': 'AM', 'value': 80}],
     'name': 'Taste/Odor, Chlorine (QA1)',
     'value': 240},
    {'children': [{'name': 'PM', 'value': 144}, {'name': 'AM', 'value': 77}],
     'name': 'Taste/Odor, Bitter/Metallic (QA3)',
     'value': 221},
    {'children': [{'name': 'PM', 'value': 114}, {'name': 'AM', 'value'

In [16]:
import json

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [17]:
with open('json_data.json', 'w') as fp:
    json.dump(d0, fp, cls=MyEncoder)